# Cocktails und duale Preise

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/Cocktails-Duality.ipynb)

- [Cocktails (shared)](https://febunisofia-my.sharepoint.com/:x:/g/personal/amarov_feb_uni-sofia_bg/ESP-SwOFNNhGpNlhQtnI1xsBrR8nw-eklsYqMyCA-eJiAw?e=Tfqjsq&nav=MTVfezAwMDAwMDAwLTAwMDEtMDAwMC0wMDAwLTAwMDAwMDAwMDAwMH0)
- [Cocktails (download)](https://github.com/febse/data/raw/refs/heads/main/opt/Cocktails_2d.xlsx)

In [ ]:
%pip install gurobipy plotly sympy

import numpy as np
import plotly.graph_objects as go
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Note: you may need to restart the kernel to use updated packages.


## Barmanagement

Stellen wir uns vor, dass wir eine kleine Bar managen. Wir sind so spezialisiert, dass wir nur Bloody Mary in zwei Varianten anbieten:

- Bloody Mary Light (BML), der aus 20 ml Vodka und 80 ml Tomatensaft besteht
- Bloody Mary Stark (BMS), der aus 40 ml Vodka und 60 ml Tomatensaft besteht

In einer Stunde haben 2000 ml Vodka und 4000 ml Tomatensaft zur Verfügung. Wie viele von beiden Cocktails sollten wir zubereiten, um den Gewinn zu maximieren? Ein Glass Bloody Mary Light (100 ml) bring uns 3 Euro Gewinn ein, ein Glass (100 ml) Bloody Mary Stark bringt uns 4 Euro Gewinn ein.

- Schreiben Sie das Modell zuerst auf Papier auf
- Lösen Sie das Modell mit Excel Solver
- Lösen Sie das Modell mit Gurobi in Python (einmal mit `.addVar` und einmal mit `.addMVar`)
- Formulieren Sie die duale Optimierungsaufgabe und lösen Sie diese mit Excel Solver und Gurobi (auch mit `.addVar` und `.addMVar`)

In [ ]:
ressourcen = np.array([2000, 4000]) # ml Vodka, ml Tomatensaft

print("Ressourcen:", ressourcen)

gewinn_pro_coktail = np.array([3, 4])

print("Gewinn pro Cocktail:", gewinn_pro_coktail)

rezept_BML = np.array([20, 80]) # ml Vodka, ml Tomatensaft
rezept_BMS = np.array([40, 60]) # ml Vodka, ml Tomatensaft

bar_rezepte = np.array([rezept_BML, rezept_BMS])
bar_rezepte

Ressourcen: [2000 4000]
Gewinn pro Cocktail: [3 4]


array([[20, 80],
       [40, 60]])

In [5]:
verbrauchs_koeffizienten = bar_rezepte.T
verbrauchs_koeffizienten

array([[20, 40],
       [80, 60]])

In [12]:
verbrauch_vodka = verbrauchs_koeffizienten[0,:]
print("Verbrauch von Vodka:", verbrauch_vodka)

verbrauch_tomatensaft = verbrauchs_koeffizienten[1,:]
print("Verbrauch von Tomatensaft:", verbrauch_tomatensaft)


Verbrauch von Vodka: [20 40]
Verbrauch von Tomatensaft: [80 60]


Berechnen Sie den Verbrauch von Vodka und Tomatensaft für die Produktion von 

- 2 Cocktails BML und 3 Cocktails BMS
- 3 Cocktails BML und 2 Cocktails BMS
- 4 Cocktails BML und 1 Cocktails BMS
- 1 Cocktails BML und 4 Cocktails BMS

Benutzen Sie dazu Formeln in Excel und `numpy` in Python

In [ ]:
# Verbrauch von Vodka bei der Produktion von 2 Bloody Mary Light und 3 Bloody Mary Stark
verbrauch_vodka @ np.array([2, 3])

# Verbrauch von Tomatensaft bei der Produktion von 2 Bloody Mary Light und 3 Bloody Mary Stark


np.int64(160)

In [ ]:
# Modell erstellen

m = gp.Model("Cocktail")
m.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

# Variablen definieren

x1 = m.addVar(name="Light") # Anzahl von Bloody Mary Light Cocktails
x2 = m.addVar(name="Stark") # Anzahl von Bloody Mary Stark Cocktails

# Zielfunktion definieren

m.setObjective(3 * x1 + 4 * x2, GRB.MAXIMIZE)

# Nebenbedingungen definieren

m.addConstr(20 * x1 + 40 * x2 <= ressourcen[0], "Vodka")
m.addConstr(80 * x1 + 60 * x2 <= ressourcen[1], "Tomatensaft")

m.optimize()

m.write("cocktail.lp") # Speichern des Modells in einer Datei

with open("cocktail.lp", "r") as f:
    print(f.read())

# Ergebnisse ausgeben

print("Optimaler Cocktailmix:")

print(f"Bloody Mary Light: {x1.X:.2f} Stück")
print(f"Bloody Mary Stark: {x2.X:.2f} Stück")

# Die Nebenbedingungen ausgeben als pandas DataFrame

# Die Nebenbedingungen ausgeben als pandas DataFrame
constr_df = pd.DataFrame([
    (c.ConstrName, c.Pi, c.Slack, c.RHS) for c in m.getConstrs()],
    columns=["Name", "Schattenpreis", "Slack", "RHS"]
)
constr_df

Set parameter LogToConsole to value 0


\ Model Cocktail
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  3 Light + 4 Stark
Subject To
 Vodka: 20 Light + 40 Stark <= 2000
 Tomatensaft: 80 Light + 60 Stark <= 4000
Bounds
End

Optimaler Cocktailmix:
Bloody Mary Light: 20.00 Stück
Bloody Mary Stark: 40.00 Stück


,Name,Slack,RHS,Schattenpreis,Sense
0,Vodka,0.0,2000.0,0.07,<
1,Tomatensaft,0.0,4000.0,0.02,<


In [ ]:
# Dasselbe in kürzerer Form

m1 = gp.Model("Cocktail 2")
m1.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

x = m1.addMVar(2, name="Cocktails")

m1.setObjective(gewinn_pro_coktail @ x, GRB.MAXIMIZE)

m1.addConstr(verbrauchs_koeffizienten @ x <= ressourcen, "Ressourcen")

m1.optimize()

# Ergebnisse ausgeben
print("Optimaler Cocktailmix:")
print(f"Bloody Mary Light: {x.X[0]:.2f} Stück")
print(f"Bloody Mary Stark: {x.X[1]:.2f} Stück")

m1.write("cocktail-2.lp")

with open("cocktail-2.lp") as f:
    print(f.read())

# Die Nebenbedingungen ausgeben als pandas DataFrame
constr_df = pd.DataFrame([
    (c.ConstrName, c.Pi, c.Slack, c.RHS) for c in m1.getConstrs()],
    columns=["Name", "Schattenpreis", "Slack", "RHS"]
)
constr_df

Set parameter LogToConsole to value 0
Optimaler Cocktailmix:
Bloody Mary Light: 20.00 Stück
Bloody Mary Stark: 40.00 Stück
\ Model Cocktail 2
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  3 Cocktail[0] + 4 Cocktail[1]
Subject To
 Rezept[0]: 20 Cocktail[0] + 40 Cocktail[1] <= 2000
 Rezept[1]: 80 Cocktail[0] + 60 Cocktail[1] <= 4000
Bounds
End



,Name,Slack,RHS,Schattenpreis,Sense
0,Rezept[0],0.0,2000.0,0.07,<
1,Rezept[1],0.0,4000.0,0.02,<


## Das duale Problem

Schauenf wir uns eine einfache Optimierungsaufgabe an:

$$
\begin{align*}
\text{max} & \quad 2x_1 + 3x_2 \\
\text{s.t.} & \quad x_1 + x_2 \leq 8 \\
& \quad x_1 + 2 x_2 \leq 12 \\
& x_1, x_2 \geq 0
\end{align*}
$$

Ihre Lösung ist $[x_1, x_2] = [4, 4]$, also erreichen wir den grösstmöglichen Gewinn von $2 \cdot 4 + 3 \cdot 4 = 20$ Euro. Können wir aber **beweisen**, dass es keinen grösseren Gewinn gibt?
 



Die Optimierungsaufgabe für die kleine Bar lautet:

$$
\begin{align*}
\text{max} & \quad 3x_1 + 4x_2 \\
\text{s.t.} & \quad 20 x_1 + 40x_2 \leq 2000 \\
& \quad 80 x_1 + 60x_2 \leq 4000 \\
& \quad x_1, x_2 \geq 0
\end{align*}
$$

wobei $x_1$ und $x_2$ die Anzahl der Cocktails Bloody Mary Light und Bloody Mary Stark darstellen. Wir haben auch den optimalen Produktionsplan $[x_1, x_2] = [20, 40]$, also erreichen wir den grösstmöglichen Gewinn von $3 \cdot 20 + 4 \cdot 40 = 220$ Euro, indem wir 20 Cocktails Bloody Mary Light und 40 Cocktails Bloody Mary Stark zubereiten. Können wir aber **beweisen**, dass dies der optimale Produktionsplan ist?

Lassen Sie uns die zwei Ungleichungen mit den Faktoren $0.07$ und $0.02$ multiplizieren und dann addieren:

$$
\begin{align*}
0.07 \cdot (20 x_1 + 40x_2) + 0.02 \cdot (80 x_1 + 60x_2) & \leq 0.07 \cdot 2000 + 0.02 \cdot 4000 \\
\end{align*}
$$

Jetzt bringen wir diese Gleichung in die Form:

$$
\begin{align*}
(0.07 \cdot 20 + 0.02 \cdot 80) x_1 + (0.07 \cdot 40 + 0.02 \cdot 60) x_2 & \leq 0.07 \cdot 2000 + 0.02 \cdot 4000 
\end{align*}
$$

Also haben wir:

$$
\begin{align*}
3 x_1 + 4 x_2 & \leq 220 \\
\end{align*}
$$

Nun haben wir eine neue Ungleichung, in der auf der linken Seite eigentlich die Zielfunktion steht. Diese Ungleichung sagt uns aber, dass unter den gegebenen Bedingungen der Gewinn nicht grösser als 220 Euro sein kann, was ein Beweis für die Optimalität unseres Produktionsplans ist.

Diese Ungleichung haben wir aus den ursprünglichen Ungleichungen abgeleitet, die wir mit den Faktoren $0.07$ und $0.02$ multipliziert haben. Wie haben wir diese Faktoren gefunden? Lassen Sie uns die ursprüngliche Aufgabe noch einmal aufschreiben:

$$
\begin{align*}
\text{max} & \quad 3x_1 + 4x_2 \\
\text{s.t.} & \quad 20 x_1 + 40x_2 \leq 2000 \\
& \quad 80 x_1 + 60x_2 \leq 4000 \\
& \quad x_1, x_2 \geq 0
\end{align*}
$$

Wir können beide Ungleichungen mit irgendwelchen Faktoren $y_1$ und $y_2$ multiplizieren. Solange $y_1$ und $y_2$ positiv sind, ändert sich die Ungleichung nicht. Wenn wir die beiden Ungleichungen addieren, erhalten wir eine neue Ungleichung:

$$
\begin{align*}
(20 x_1 + 40x_2)y_1 & \leq 2000y_1 \\
(80 x_1 + 60x_2)y_2 & \leq 4000y_2 \\
\end{align*}
$$

$$
\begin{align*}
20 y_1 x_1 + 40 y_1 x_2 & \leq 2000 y_1 \\
80 y_2 x_1 + 60 y_2 x_2 & \leq 4000 y_2 \\
\end{align*}
$$

Nun addieren wir die beiden Ungleichungen:

$$
\begin{align*}
(20 y_1 + 80 y_2) x_1 + (40 y_1 + 60 y_2) x_2 & \leq 2000 y_1 + 4000 y_2 \\
\end{align*}
$$

Wie linke Seite der Ungleichung hat die Form der Zielfunktion, nur dass die Variablen $x_1$ und $x_2$ mit den Faktoren $20 y_1 + 80 y_2$ und $40 y_1 + 60 y_2$ multipliziert werden und nicht mit den Faktoren $3$ und $4$. Falls wir aber fordern, dass die Faktoren $20 y_1 + 80 y_2$ und $40 y_1 + 60 y_2$ grösser oder gleich $3$ und $4$ sind, dann haben wir eine neue Ungleichung, die uns sagt, die linke Seite immer grösser oder gleich der Zielfunktion ist. 

Schreiben wir die Ungleichungen auf:

$$
\begin{align*}
\underset{\text{Zielfunktion}}{3 x_1 + 4 x_2} & \leq (20 y_1 + 80 y_2) x_1 + (40 y_1 + 60 y_2) x_2 \leq \underset{\text{Obere Schranke}}{2000 y_1 + 4000 y_2} \\
\end{align*}
$$

$$
\begin{align*}
20 y_1 + 80 y_2 & \geq 3 \\
40 y_1 + 60 y_2 & \geq 4 \\
\end{align*}
$$

Interessant ist hier, dass wir eine obere Schranke für die Zielfunktion hergeleitet haben. Nun stellt sich die Frage, ob wir die **kleinste** obere Schranke finden können. Die obere Schranke ist linear, die Einschränkungen sind auch linear, also haben wir nur noch eine neue lineare Optimierungsaufgabe zu lösen:

$$
\begin{align*}
\text{min} & \quad 2000 y_1 + 4000 y_2 \\
\text{s.t.} & \quad 20 y_1 + 80 y_2 \geq 3 \\
& \quad 40 y_1 + 60 y_2 \geq 4 \\
& \quad y_1, y_2 \geq 0 \\
\end{align*}
$$

Bevor wir mit der Lösung anfangen, ist es sinnvoll, sich die Messeinheiten der $y_1$ und $y_2$ Variablen anzuschauen.

$$
\underset{\text{Gewinn (Euro)}}{3 x_1 + 4 x_2} \leq \underset{\text{Obere Schranke (Euro)}}{2000 y_1 + 4000y_2}
$$

Damit die Messeinheiten kompatibel sind, muss die rechte Seite der Ungleichung in Euro sein. Die Koeffizienten (2000 ml) und (4000 ml) sind die Mengen der Zutaten, die wir haben. Also müssen die Produkte von $2000 y_1$ und $2000 y_2$ mit in Euro gemessen sein.

Also sind die Messeinheiten von $y_1$ und $y_2$ Euro pro ml.

$$
2000 [\text{ml}] \cdot y_1 \left[\frac{\text{Euro}}{\text{ml}}\right] + 4000 [\text{ml}] \cdot y_2 \left[\frac{\text{Euro}}{\text{ml}}\right] = [\text{Euro}]
$$



In [ ]:
# Das duale Problem (1)

md = gp.Model("Cocktail dual")
md.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

y1 = md.addVar(name="Wert von Vodka") # Schattenpreis Vodka
y2 = md.addVar(name="Wert von Tomatensaft") # Schattenpreis Tomatensaft

# Zielfunktion definieren
md.setObjective(ressourcen[0] * y1 + ressourcen[1] * y2, GRB.MINIMIZE)

# Nebenbedingungen definieren

md.addConstr(20 * y1 + 80 * y2 >= 3, "Bloody Mary Light")
md.addConstr(40 * y1 + 60 * y2 >= 4, "Bloody Mary Stark")


md.write("cocktail-dual.lp") # Speichern des Modells in einer Datei

with open("cocktail-dual.lp", "r") as f:
    print(f.read())

md.optimize()

# Ergebnisse ausgeben
print("Optimale Werte der Ressourcen:")

print(f"Wert von Vodka: {y1.X:.2f} Euro /ml")
print(f"Wert von Tomatensaft: {y2.X:.2f} Euro/ml")
print(f"Gewinn: {md.ObjVal:.2f} Euro")

# Die Nebenbedingungen ausgeben als pandas DataFrame
constr_df = pd.DataFrame([
    (c.ConstrName, c.Slack, c.RHS, c.Pi, c.Sense) for c in md.getConstrs()], 
    columns=["Name", "Slack", "RHS", "Schattenpreis", "Sense"
])

constr_df


Set parameter LogToConsole to value 0
\ Model Cocktail dual
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  2000 SP_Vodka + 4000 SP_Tomatensaft
Subject To
 Bloody_Mary_Light: 20 SP_Vodka + 80 SP_Tomatensaft >= 3
 Bloody_Mary_Stark: 40 SP_Vodka + 60 SP_Tomatensaft >= 4
Bounds
End

Optimale Schattenpreise:
Vodka: 0.07 Euro /ml
Tomatensaft: 0.02 Euro/ml
Gewinn: 220.00 Euro


,Name,Slack,RHS,Schattenpreis,Sense
0,Bloody Mary Light,0.0,3.0,20.0,>
1,Bloody Mary Stark,0.0,4.0,40.0,>


In [16]:
# Das Duale Problem (2)

m2 = gp.Model("Cocktail Dual 2")

m2.Params.LogToConsole = 0 # keine Ausgabe in der Konsole

# Variablen definieren
y = m2.addMVar(2, name="Werte")

# Zielfunktion definieren
m2.setObjective(ressourcen @ y, GRB.MINIMIZE)

# Nebenbedingungen definieren

m2.addConstr(bar_rezepte @ y >= gewinn_pro_coktail, "Cocktail")

m2.optimize()

m2.write("cocktail-dual-2.lp") # Speichern des Modells in einer Datei

with open("cocktail-dual-2.lp", "r") as f:
    print(f.read())

# Ergebnisse ausgeben
print("Schattenpreise:")

print(f"Wert von Vodka: {y.X[0]:.2f} Euro/ml")
print(f"Wert von Tomatensaft: {y.X[1]:.2f} Euro/ml")
print(f"Gewinn: {m2.ObjVal:.2f} Euro")
# Die Nebenbedingungen ausgeben als pandas DataFrame

constr_df = pd.DataFrame([
    (c.ConstrName, c.Slack, c.RHS, c.Pi, c.Sense) for c in m2.getConstrs()],
    columns=["Name", "Slack", "RHS", "Schattenpreis", "Sense"
])

constr_df


Set parameter LogToConsole to value 0
\ Model Cocktail Dual 2
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  2000 Werte[0] + 4000 Werte[1]
Subject To
 Cocktail[0]: 20 Werte[0] + 80 Werte[1] >= 3
 Cocktail[1]: 40 Werte[0] + 60 Werte[1] >= 4
Bounds
End

Schattenpreise:
Wert von Vodka: 0.07 Euro/ml
Wert von Tomatensaft: 0.02 Euro/ml
Gewinn: 220.00 Euro


,Name,Slack,RHS,Schattenpreis,Sense
0,Cocktail[0],0.0,3.0,20.0,>
1,Cocktail[1],0.0,4.0,40.0,>


In [ ]:
# Wie viel von den Ressourcen verbrauchen wir, falls wir 2 Bloody Marys Light und 2 Bloody Marys Stark machen?

bar_rezepte @ np.array([2, 2])

array([100, 100])

In [ ]:
# Scale in the x direction

def create_frame_data(v1, v2, A):
    v1_tr, v2_tr = A @ v1, A @ v2

    tr_sum = v1_tr + v2_tr

    return [
        go.Scatter(
            x=[0, v1_tr[0]],
            y=[0, v1_tr[1]],
            mode='lines+markers',
            name='V1',
            line=dict(color='blue', width=1),
            marker=dict(size=2)
        ),
        go.Scatter(
            x=[0, v2_tr[0]],
            y=[0, v2_tr[1]],
            mode='lines+markers',
            name='V2',
            line=dict(color='red', width=1),
            marker=dict(size=2)
        ),
        go.Scatter(
            x=[0, tr_sum[0]],
            y=[0, tr_sum[1]],
            mode='lines+markers',
            name='V1 + V2',
            line=dict(color='green', width=1),
            marker=dict(size=2)
        ),
        go.Scatter(
            x=[0, v1_tr[0], tr_sum[0], v2_tr[0], 0],  # Define the polygon vertices
            y=[0, v1_tr[1], tr_sum[1], v2_tr[1], 0],
            fill='toself',
            fillcolor='rgba(0, 100, 200, 0.2)',  # Set fill color with transparency
            line=dict(color='rgba(255,255,255,0)'),  # No border line
            name='Area'
        )
    ]

def create_animation(v1, v2, A_list):    
    # Plot the unit vectors and their sum
    A = A_list[0]

    fig = go.Figure(
        data=create_frame_data(v1, v2, A),
        layout=go.Layout(
            title='Area and stretching in the x direction',
            xaxis=dict(title='x', range=[-2.3, 2.3]),  # Set X-axis range
            yaxis=dict(title='y', range=[-2.3, 2.3]),  # Set Y-axis range
            showlegend=True,
            width=800,
            height=600
        ),
    )

    frames = [go.Frame(data = create_frame_data(v1, v2, A_frame)) for A_frame in A_list]

    fig.update_layout(width=600, height=450,
            updatemenus = [
                dict(type = "buttons",
                    buttons = [
                        dict(
                    args = [None, {"frame": {"duration": 50, "redraw": False},
                                    "fromcurrent": True, "transition": {"duration": 10}}],
                    label = "Play",
                    method = "animate",
                    )])])

    fig.update(frames=frames)
    
    return fig

unit1 = np.array([1, 0])
unit2 = np.array([0, 1])

A_list = [np.identity(2)] + [
    np.array([[x, 0], [0, 1]]) for x in np.linspace(1, 2, 50)
]

fig1 = create_animation(unit1, unit2, A_list)
fig1.show()


In [ ]:
# Scale in the y direction

fig2 = create_animation(unit1, unit2, [
    np.array([[1, 0], [0, y]]) for y in np.linspace(1, 2, 50)
])
fig2.show()

In [ ]:
# Scale in the x and y direction

fig3 = create_animation(unit1, unit2, [
    np.array([[x, 0], [0, y]]) for x, y in zip(np.linspace(1, 2, 50), np.linspace(1, 2, 50))
])
fig3.show()

In [ ]:
# Sheer in the x direction

fig4 = create_animation(unit1, unit2, [
    np.array([[1, x], [0, 1]]) for x in np.linspace(0, 1, 50)
])
fig4.show()

In [ ]:
# Sheer in the y direction

fig5 = create_animation(unit1, unit2, [
    np.array([[1, 0], [x, 1]]) for x in np.linspace(0, 1, 50)
])
fig5.show()


In [ ]:
# Sheer in the x and y direction

fig6 = create_animation(unit1, unit2, [
    np.array([[1, x], [y, 1]]) for x, y in zip(np.linspace(0, 1, 50), np.linspace(0, 1, 50))
])
fig6.show()


In [ ]:
# Scale and shear in the in both directions

fig7 = create_animation(unit1, unit2, [
    np.array([[x, x1], [y1, y]]) for x, y, x1, y1 in zip(np.linspace(1, 2, 50), np.linspace(1, 2, 50), np.linspace(0, 1, 50), np.linspace(0, 1, 50))])
fig7.show()

In [ ]:
np.ones((2, 2)) @ np.array([-2, 5])

array([3., 3.])

In [ ]:
np.ones((2, 2)) @ np.array([-1, 1])

array([0., 0.])

In [ ]:
np.linalg.det(np.ones((2, 2)))

np.float64(0.0)

In [ ]:
# Scale in the x direction

def create_frame_data1(v1, v2, A):
    v1_tr, v2_tr = A @ v1, v2

    return [
        go.Scatter(
            x=[0, v1_tr[0]],
            y=[0, v1_tr[1]],
            mode='lines+markers',
            name='V1',
            line=dict(color='blue', width=1),
            marker=dict(size=2)
        ),
        go.Scatter(
            x=[0, v1[0]],
            y=[0, v1[1]],
            mode='lines+markers',
            name='V1',
            line=dict(color='blue', width=1, dash='dash'),
            marker=dict(size=2)
        ),
        go.Scatter(
            x=[0, v2_tr[0]],
            y=[0, v2_tr[1]],
            mode='lines+markers',
            name='V2',
            line=dict(color='red', width=1),
            marker=dict(size=2)
        ),
    ]

def create_animation1(v1_list, v2, A, xrange=[-2.3, 2.3], yrange=[-2.3, 2.3], title='Area and stretching in the x direction'):    
    # Plot the unit vectors and their sum
    fig = go.Figure(
        data=create_frame_data1(v1_list[0], v2, A),
        layout=go.Layout(
            title=title,
            xaxis=dict(title='x', range=xrange),  # Set X-axis range
            yaxis=dict(title='y', range=yrange),  # Set Y-axis range
            showlegend=True,
            width=800,
            height=600
        ),
    )

    frames = [go.Frame(data = create_frame_data1(v1, v2, A)) for v1 in v1_list]

    fig.update_layout(width=600, height=450,
            updatemenus = [
                dict(type = "buttons",
                    buttons = [
                        dict(
                    args = [None, {"frame": {"duration": 50, "redraw": False},
                                    "fromcurrent": True, "transition": {"duration": 10}}],
                    label = "Play",
                    method = "animate",
                    )])])

    fig.update(frames=frames)
    
    return fig


v2 = np.array([-1, 4])

A = np.array([[2, -1], [1, 3]])

sol = np.linalg.solve(A, v2)

v1_list = [np.array([v1_, v2_]) for v1_, v2_ in zip(np.linspace(-2, sol[0], 50), np.linspace(0, sol[1], 50))]

fig9 = create_animation1(v1_list, v2, A, xrange=[-6, 1], yrange=[-3, 5], title='Solution of a system of equations')
fig9.show()


# Cocktails mit drei Zutaten


In [ ]:
# Ingredients 100 ml vodka, 100 ml rum, 100 ml tomato juice
zutaten_3 = np.array([1, 1, 1])

# Rezepte (in ml)

bloody_mary_rezept = np.array([20, 0, 80])
rum_pure_rezept = np.array([0, 50, 0])
crazy_rezept = np.array([40, 40, 20])

alle_rezepte = np.stack([
    bloody_mary_rezept,
    rum_pure_rezept,
    crazy_rezept
])

rezept_namen = [
    "Bloody Mary Rezept",
    "Rum Pure Rezept",
    "Crazy Cocktail Rezept"
]

cocktail_colors = [
    "firebrick",
    "steelblue",
    "orange"
]

# Create a 3D scatter plot with plotly showing the rows of all_cocktails as vectors 

fig = go.Figure()

# Add each cocktail as a vector
for i, cocktail in enumerate(alle_rezepte):
    fig.add_trace(go.Scatter3d(
        x=[0, cocktail[0]],  # Start at origin (0, 0, 0)
        y=[0, cocktail[1]],
        z=[0, cocktail[2]],
        mode='lines+markers',
        name=rezept_namen[i],
        line=dict(color=cocktail_colors[i], width=5),
        marker=dict(size=5)
    ))

# Set plot layout
fig.update_layout(
    title="3D Diagram der Cocktails",
    scene=dict(
        xaxis=dict(
            title="Vodka",
            range=[0, 1]
        ),
        yaxis=dict(
            title="Rum",
            range=[0, 1]
        ),
        zaxis=dict(
            title="Tomatensaft",
            range=[0, 1]
        )
    )
)

# Show the plot
fig.show()

## Modifikation von Rezepten


In [ ]:

# Nimm die Zutaten, erhöhe die erste Zutat (Vodka) um 20 Prozent, ignoriere alle anderen Zutaten
nimm_den_vodka_verdoppele_ihn_und_ignoriere_den_rest = np.array([2, 0, 0])

# Nimm die zweite Zutat (Rum), verändere die Menge nicht, ignoriere alle anderen Zutaten
nimm_nur_den_rum_so_wie_er_ist = np.array([0, 1, 0])

# Nimm die dritte Zutat (Tomato Juice), verändere die Menge nicht, ignoriere alle anderen Zutaten
nimm_den_tomatensaft_so_wie_er_ist = np.array([0, 0, 1])

In [ ]:
verdoppele_den_vodka_im_rezept = np.stack([
    nimm_den_vodka_verdoppele_ihn_und_ignoriere_den_rest,
    nimm_nur_den_rum_so_wie_er_ist,
    nimm_den_tomatensaft_so_wie_er_ist
])

verdoppele_den_vodka_im_rezept

array([[2, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [ ]:
modifizierte_rezepte = verdoppele_den_vodka_im_rezept @ alle_rezepte.T
print(alle_rezepte)
modifizierte_rezepte.T

[[0.3 0.  0.6]
 [0.  0.4 0. ]
 [0.4 0.4 0.3]]


array([[0.6, 0. , 0.6],
       [0. , 0.4, 0. ],
       [0.8, 0.4, 0.3]])

In [ ]:
# Create a 3D scatter plot with plotly showing the rows of all_cocktails as vectors 

# Add each cocktail as a vector
for i, cocktail in enumerate(modifizierte_rezepte.T):
    fig.add_trace(go.Scatter3d(
        x=[0, cocktail[0]],  # Start at origin (0, 0, 0)
        y=[0, cocktail[1]],
        z=[0, cocktail[2]],
        mode='lines+markers',
        name=f"{rezept_namen[i]} modifiziert",
        line=dict(color=cocktail_colors[i], width=5, dash='dash'),
        marker=dict(size=5)
    ))

fig.update_layout(
    title = "3D Diagram der Cocktails: die originalen Rezepte und die modifizierten Rezepte",
)
# Show the plot
fig.show()

In [ ]:
bloody_mary_mit_doppelt_vodka_rezept = verdoppele_den_vodka_im_rezept @ bloody_mary_rezept
bloody_mary_mit_doppelt_vodka_rezept

array([0.6, 0. , 0.6])

In [ ]:
crazy_mit_doppelt_vodka_rezept = crazy_rezept @ verdoppele_den_vodka_im_rezept
crazy_mit_doppelt_vodka_rezept

array([0.8, 0.4, 0.3])

In [ ]:
switch_vodka_and_tomato = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
])
switch_vodka_and_tomato

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])